# Model Performance Analytics

## Imports

In [1]:
from typing import Tuple

import boto3 as aws
import pandas as pd
import seaborn as sns
from botocore.exceptions import ClientError

## Load Data

In [19]:
def download_metrics(aws_bucket: str) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """Get all model and test metrics from AWS S3 bucket.
    
    This function retreives all model and test metrics that have been
    persisted to-date and returns them as two DataFrames.
    """
    def _load_dataset_from_aws_s3(s3_obj_key: str) -> pd.DataFrame:
        """Load CSV datafile from AWS S3 into DataFrame."""
        object_data = s3_client.get_object(
            Bucket=aws_bucket,
            Key=s3_obj_key
        )
        return pd.read_csv(object_data['Body'])

    try:
        s3_client = aws.client('s3')
        print(f'downloading model metrics from s3://{aws_bucket}/model-metrics')
        s3_model_metrics_objects = s3_client.list_objects(
            Bucket=aws_bucket,
            Prefix='model-metrics/'
        )
        model_metrics = pd.concat(
            _load_dataset_from_aws_s3(obj['Key'])
            for obj in s3_model_metrics_objects['Contents']
        )
        print(f'downloading test metrics from s3://{aws_bucket}/model-metrics')
        s3_test_metrics_objects = s3_client.list_objects(
            Bucket=aws_bucket,
            Prefix='test-metrics/'
        )
        test_metrics = pd.concat(
            _load_dataset_from_aws_s3(obj['Key'])
            for obj in s3_test_metrics_objects['Contents']
        )
        test_metrics.reset_index(drop=True, inplace=True)
    except ClientError as e:
        print(f'failed to download training data from s3://{aws_bucket}/datasets')
    return (model_metrics, test_metrics)

Get the metrics!

In [20]:
model_metrics, test_metrics = download_metrics('bodywork-ml-ops-project')

downloading model metrics from s3://bodywork-ml-ops-project/model-metrics
downloading test metrics from s3://bodywork-ml-ops-project/model-metrics


In [18]:
model_metrics

,date,MAPE,r_squared,max_residual
0,2021-01-23,0.191461,0.651811,30.776237
1,2021-01-24,0.170312,0.639041,31.681364
2,2021-01-25,0.181855,0.671631,33.556766
3,2021-01-26,0.173301,0.657730,29.831585
4,2021-01-27,0.177455,0.657632,34.994000
5,2021-01-28,0.176515,0.665388,34.528618
6,2021-01-29,0.187144,0.675043,31.603913
7,2021-01-30,0.175229,0.673203,34.143840
8,2021-01-31,0.174100,0.673762,34.181483
9,2021-02-01,0.176044,0.675337,33.868469
